In [5]:
import pandas as pd
import numpy as np
import glob
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from joblib import dump
import warnings
import json
warnings.filterwarnings('ignore')

def convert_to_serializable(obj):
    if isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    elif isinstance(obj, (pd.DataFrame,)):
        return obj.to_dict('records')
    elif isinstance(obj, (pd.Series,)):
        return obj.to_dict()
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    else:
        return obj

# Set data paths
flight_data_path = './cleaned_data/'
weather_data_path = './cleaned_weather_data/'
top_airports_file = './top_100_airports.csv'
output_dir = './dep_delay_lr/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print("Starting year-by-year flight delay prediction models (Logistic Regression)...")
print(f"Flight data directory: {flight_data_path}")
print(f"Weather data directory: {weather_data_path}")
print(f"Top airports file: {top_airports_file}")
print(f"Model output directory: {output_dir}")

# Load top 30 airports from the top 100 airports file
try:
    top_airports = pd.read_csv(top_airports_file, low_memory=False)
    
    top_airports = top_airports.head(30)
    
    top_airport_codes = set(top_airports['ORIGIN_IATA'].str.strip().tolist())
    
    print(f"Loaded top 30 airports: {', '.join(sorted(top_airport_codes))}")
    print(f"Busiest airport: {top_airports.iloc[0]['ORIGIN_IATA']} with {top_airports.iloc[0]['Times']} flights")
    print(f"30th busiest airport: {top_airports.iloc[29]['ORIGIN_IATA']} with {top_airports.iloc[29]['Times']} flights")
except Exception as e:
    print(f"Error loading top airports file: {e}")
    top_airport_codes = None
    print("Will process all airports (top airports file not available)")

Starting year-by-year flight delay prediction models (Logistic Regression)...
Flight data directory: ./cleaned_data/
Weather data directory: ./cleaned_weather_data/
Top airports file: ./top_100_airports.csv
Model output directory: ./dep_delay_lr/
Loaded top 30 airports: ATL, AUS, BNA, BOS, BWI, CLT, DCA, DEN, DFW, DTW, EWR, FLL, IAD, IAH, JFK, LAS, LAX, LGA, MCO, MDW, MIA, MSP, ORD, PHL, PHX, SAN, SEA, SFO, SLC, TPA
Busiest airport: ATL with 457121 flights
30th busiest airport: TPA with 97235 flights


In [6]:
# Function to load weather data
def load_weather_data():
    print("\nLoading weather data...")
    start_time = time.time()
    
    all_files = glob.glob(os.path.join(weather_data_path, "*.csv"))
    print(f"Found {len(all_files)} total weather data files")
    weather_dict = {}
    count = 0
    matching_count = 0

    for file in all_files:
        try:
            filename = os.path.basename(file)
            parts = filename.split('.')[0].split('_')
            
            if len(parts) >= 3:
                iata = parts[0]
                year = parts[1]
                month_name = parts[2]
                
                # Convert month name to number
                month_map = {
                    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                    'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
                }
                
                if month_name in month_map:
                    month = month_map[month_name]
                    
                    if top_airport_codes is None or iata in top_airport_codes:
                        weather_data = pd.read_csv(file, low_memory=False)

                        if 'DATE' not in weather_data.columns:
                            print(f"Warning: DATE column not found in {filename}")
                            continue

                        weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

                        key = f"{iata}_{year}_{month}"

                        weather_dict[key] = weather_data
                        matching_count += 1
                else:
                    print(f"Warning: Unknown month format in {filename}")
                
                count += 1

                if count % 100 == 0:
                    print(f"Processed {count} weather files, loaded {matching_count} matching files")
        except Exception as e:
            print(f"Error loading weather file {file}: {e}")
    
    print(f"Loaded {matching_count} weather files out of {count} processed files")
    print(f"Loading weather data took: {time.time() - start_time:.2f} seconds")
    return weather_dict

def get_may_files():
    may_files = [
        os.path.join(flight_data_path, "May2021.csv"),
        os.path.join(flight_data_path, "May2022.csv"),
        os.path.join(flight_data_path, "May2023.csv"),
        os.path.join(flight_data_path, "May2024.csv")
    ]

    existing_files = []
    for file_path in may_files:
        if os.path.exists(file_path):
            existing_files.append(file_path)
        else:
            print(f"Warning: File {file_path} not found")
    
    return existing_files

flight_files = get_may_files()
print(f"\nFound {len(flight_files)} May files to process:")
for f in flight_files:
    print(f"  - {os.path.basename(f)}")

if not flight_files:
    print("No May 2021-2024 files were found. Please check file paths.")
    exit(1)

# Load all weather data once
weather_dict = load_weather_data()

# Function to extract year from filename
def extract_year_from_filename(filename):
    base_name = os.path.basename(filename)
    year_str = base_name.replace('May', '').split('.')[0]
    return int(year_str)

# Function to create red-eye flight indicator
def create_redeye_indicator(df):
    # Make a copy
    df = df.copy()

    df['IS_REDEYE'] = 0
    time_columns = []

    if 'SCH_DEP_TIME' in df.columns:
        time_columns.append('SCH_DEP_TIME')

    if 'SCH_ARR_TIME' in df.columns:
        time_columns.append('SCH_ARR_TIME')

    for col in time_columns:
        if df[col].dtype != 'float64':
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                print(f"Warning: Could not convert {col} to numeric")
    
    # Identify red-eye flights
    if 'SCH_DEP_TIME' in time_columns:
        redeye_departure = (df['SCH_DEP_TIME'] >= 0) & (df['SCH_DEP_TIME'] < 600)
        df.loc[redeye_departure, 'IS_REDEYE'] = 1
        
        # Count departures identified as red-eye
        dep_redeye_count = redeye_departure.sum()
        print(f"Identified {dep_redeye_count} red-eye flights based on departure time (0-6 AM)")

    if 'SCH_ARR_TIME' in time_columns:
        redeye_arrival = (df['SCH_ARR_TIME'] >= 0) & (df['SCH_ARR_TIME'] < 600)
        df.loc[redeye_arrival, 'IS_REDEYE'] = 1
        
        # Count arrivals identified as red-eye
        arr_redeye_count = redeye_arrival.sum()
        print(f"Identified {arr_redeye_count} red-eye flights based on arrival time (0-6 AM)")

    redeye_count = df['IS_REDEYE'].sum()
    total_count = len(df)
    print(f"Total identified red-eye flights: {redeye_count} out of {total_count} total flights ({redeye_count/total_count*100:.2f}%)")

    if 'SCH_DEP_TIME' in time_columns:
        df['DEP_TIME_OF_DAY'] = pd.cut(
            df['SCH_DEP_TIME'], 
            bins=[0, 600, 1200, 1800, 2400],
            labels=['Early Morning (0-6)', 'Morning (6-12)', 'Afternoon (12-18)', 'Evening (18-24)'],
            include_lowest=True
        )

        time_dist = df['DEP_TIME_OF_DAY'].value_counts()
        print("\nDistribution of flights by departure time of day:")
        for time_cat, count in time_dist.items():
            print(f"  - {time_cat}: {count} flights ({count/total_count*100:.2f}%)")
    
    return df

# Function to prepare departure delay data
def prepare_delay_data(df):
    df = df.copy()

    if 'DEP_DELAY' in df.columns:
        if df['DEP_DELAY'].dtype != 'float64':
            try:
                df['DEP_DELAY'] = pd.to_numeric(df['DEP_DELAY'], errors='coerce')
            except:
                print(f"Warning: Could not convert DEP_DELAY to numeric")
    else:
        print("Warning: DEP_DELAY column not found in dataset")
        return df

    df['IS_DELAYED'] = (df['DEP_DELAY'] > 0).astype(int)
    
    # Create a categorical delay feature
    df['DELAY_CATEGORY'] = pd.cut(
        df['DEP_DELAY'],
        bins=[-float('inf'), -15, 0, 15, 60, 120, float('inf')],
        labels=['Very Early', 'Early', 'On Time', 'Moderate Delay',
                'Significant Delay', 'Severe Delay'],
        include_lowest=True
    )

    df['ABS_DELAY'] = np.abs(df['DEP_DELAY'])
    
    # Print delay statistics
    delay_count = df['IS_DELAYED'].sum()
    total_count = len(df)
    delay_rate = delay_count / total_count * 100
    
    print(f"\nDelay statistics:")
    print(f"Delayed flights: {delay_count}/{total_count} ({delay_rate:.2f}%)")
    print(f"On-time or early flights: {total_count - delay_count}/{total_count} ({100 - delay_rate:.2f}%)")
    
    print("\nDelay magnitude statistics:")
    print(f"Mean delay: {df['DEP_DELAY'].mean():.2f} minutes")
    print(f"Median delay: {df['DEP_DELAY'].median():.2f} minutes")
    print(f"Min delay: {df['DEP_DELAY'].min():.2f} minutes (negative means early departure)")
    print(f"Max delay: {df['DEP_DELAY'].max():.2f} minutes")

    delay_cat_dist = df['DELAY_CATEGORY'].value_counts()
    print("\nDelay category distribution:")
    for cat, count in delay_cat_dist.sort_index().items():
        print(f"  - {cat}: {count} flights ({count/total_count*100:.2f}%)")
    
    return df

# Function to create time block features
def create_time_block_features(df):
    df = df.copy()
    
    if 'SCH_DEP_TIME' not in df.columns:
        print("Warning: SCH_DEP_TIME column not found for time block features")
        return df

    if df['SCH_DEP_TIME'].dtype != 'float64':
        try:
            df['SCH_DEP_TIME'] = pd.to_numeric(df['SCH_DEP_TIME'], errors='coerce')
        except:
            print(f"Warning: Could not convert SCH_DEP_TIME to numeric")
            return df

    df['DEP_HOUR'] = (df['SCH_DEP_TIME'] / 100).astype(int)

    time_blocks = {
        0: 'Late Night (0-3)',
        1: 'Late Night (0-3)',
        2: 'Late Night (0-3)',
        3: 'Early Morning (3-6)',
        4: 'Early Morning (3-6)',
        5: 'Early Morning (3-6)',
        6: 'Morning (6-9)',
        7: 'Morning (6-9)',
        8: 'Morning (6-9)',
        9: 'Mid-Day (9-12)',
        10: 'Mid-Day (9-12)',
        11: 'Mid-Day (9-12)',
        12: 'Afternoon (12-15)',
        13: 'Afternoon (12-15)',
        14: 'Afternoon (12-15)',
        15: 'Evening (15-18)',
        16: 'Evening (15-18)',
        17: 'Evening (15-18)',
        18: 'Night (18-21)',
        19: 'Night (18-21)',
        20: 'Night (18-21)',
        21: 'Late Night (21-24)',
        22: 'Late Night (21-24)',
        23: 'Late Night (21-24)'
    }

    df['TIME_BLOCK'] = df['DEP_HOUR'].map(time_blocks)
    
    # Morning peak (7-9 AM)
    df['IS_MORNING_PEAK'] = ((df['DEP_HOUR'] >= 7) & (df['DEP_HOUR'] <= 9)).astype(int)
    
    # Evening peak (4-7 PM)
    df['IS_EVENING_PEAK'] = ((df['DEP_HOUR'] >= 16) & (df['DEP_HOUR'] <= 19)).astype(int)
    
    return df

# Function to create day of week features
def create_day_features(df):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Check if we have the WEEK column with text day names
    if 'WEEK' in df.columns:
        # Create a mapping from abbreviated day names to full day names
        day_name_map = {
            'Sun': 'Sunday',
            'Mon': 'Monday',
            'Tue': 'Tuesday',
            'Wed': 'Wednesday',
            'Thu': 'Thursday',
            'Fri': 'Friday',
            'Sat': 'Saturday'
        }
        
        # Map abbreviated names to full names
        df['DAY_NAME'] = df['WEEK'].map(day_name_map)
        
        # Create weekend indicator
        df['IS_WEEKEND'] = df['WEEK'].isin(['Sat', 'Sun']).astype(int)

        day_counts = df['DAY_NAME'].value_counts()
        total = len(df)
        print("\nDistribution of flights by day of week:")
        for day, count in day_counts.items():
            print(f"  - {day}: {count} flights ({count/total*100:.2f}%)")

        weekend_count = df['IS_WEEKEND'].sum()
        weekday_count = total - weekend_count
        print(f"\nWeekend flights: {weekend_count} ({weekend_count/total*100:.2f}%)")
        print(f"Weekday flights: {weekday_count} ({weekday_count/total*100:.2f}%)")
        
    elif 'DAY_OF_WEEK' in df.columns:
        max_day = df['DAY_OF_WEEK'].max()
        
        if max_day == 7:
            df['IS_WEEKEND'] = ((df['DAY_OF_WEEK'] == 6) | (df['DAY_OF_WEEK'] == 7)).astype(int)

            day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 
                        4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
        else:
            df['IS_WEEKEND'] = ((df['DAY_OF_WEEK'] == 5) | (df['DAY_OF_WEEK'] == 6)).astype(int)

            day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 
                        3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
        
        df['DAY_NAME'] = df['DAY_OF_WEEK'].map(day_names)
    else:
        print("Warning: No day of week column (WEEK or DAY_OF_WEEK) found")
    
    return df

# Function to load and preprocess a single flight data file
def load_and_process_flight_data(file_path):
    print(f"\nProcessing {os.path.basename(file_path)}...")
    start_time = time.time()
    
    try:
        df = pd.read_csv(file_path, low_memory=False)
        original_size = len(df)

        file_year = extract_year_from_filename(file_path)
        
        if 'YEAR' in df.columns:
            unique_years = df['YEAR'].unique()
            print(f"Years found in data: {unique_years}")

            if len(unique_years) > 1:
                df = df[df['YEAR'] == file_year]
                print(f"Filtered to only year {file_year}: {len(df)} rows")
        else:
            df['YEAR'] = file_year
            print(f"Added YEAR column with value {file_year}")

        if 'MONTH' in df.columns:
            month_counts = df['MONTH'].value_counts()
            print(f"Months found in data: {dict(month_counts)}")
            
            if 5 in month_counts:
                df = df[df['MONTH'] == 5]
                print(f"Filtered to only May data: {len(df)} rows")
            else:
                print(f"Warning: No May data found in file, but proceeding anyway as this should be May data based on filename")

        if 'DEP_DELAY' not in df.columns:
            print(f"DEP_DELAY column not found in {os.path.basename(file_path)}. Skipping file.")
            return None

        if top_airport_codes is not None:
            df = df[
                df['ORIGIN_IATA'].str.strip().isin(top_airport_codes) & 
                df['DEST_IATA'].str.strip().isin(top_airport_codes)
            ]
            
            filtered_size = len(df)
            print(f"Filtered from {original_size} to {filtered_size} rows for top 30 airports")

            if filtered_size == 0:
                print(f"No data remaining after filtering for top 30 airports. Skipping file.")
                return None

        if 'CANCELLED' in df.columns:
            cancelled_count = df['CANCELLED'].sum()
            if cancelled_count > 0:
                df = df[df['CANCELLED'] == 0]
                print(f"Removed {cancelled_count} cancelled flights, remaining: {len(df)}")
        
        print(f"Processing took: {time.time() - start_time:.2f} seconds")
        return df
        
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {e}")
        return None

# Function to match weather data to flights
def match_weather_data(df):
    print("\nMatching weather data with flights...")
    start_time = time.time()

    date_columns_exist = all(col in df.columns for col in ['YEAR', 'MONTH', 'DAY'])
    if not date_columns_exist:
        print("Warning: Missing one or more date columns (YEAR, MONTH, DAY)")
        print("Weather data cannot be matched")
        return df

    df['FLIGHT_DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

    df['WEATHER_KEY'] = df['ORIGIN_IATA'] + '_' + df['YEAR'].astype(str) + '_' + df['MONTH'].astype(str).str.zfill(2)

    weather_columns = ['EXTREME_WEATHER', 'PRCP', 'WT01', 'WT03', 'WT04', 'WT05', 'WT08', 'WT11']
    for col in weather_columns:
        if col not in df.columns:
            df[col] = 0.0
    
    # Process in batches
    matched_count = 0
    batch_size = 10000
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch = df.iloc[start_idx:end_idx]
        
        for idx, row in batch.iterrows():
            try:
                weather_key = row['WEATHER_KEY']
                flight_date = row['FLIGHT_DATE']

                if weather_key in weather_dict:
                    weather_data = weather_dict[weather_key]

                    matching_weather = weather_data[weather_data['DATE'] == flight_date]
                    
                    if not matching_weather.empty:
                        for col in weather_columns:
                            if col in matching_weather.columns:
                                df.at[idx, col] = matching_weather[col].iloc[0]
                        matched_count += 1
            except Exception as e:
                pass
        
        # Print progress
        print(f"Processed {end_idx}/{len(df)} rows, matched {matched_count} flights with weather data")
    
    print(f"Matched weather data for {matched_count} flights ({matched_count/len(df)*100:.2f}%)")
    print(f"Weather matching took: {time.time() - start_time:.2f} seconds")
    
    return df

# Function to visualize coefficients (for logistic regression)
def plot_model_coefficients(model, feature_names, year, top_n=15, output_path=None):
    coefficients = model.coef_[0]

    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    })

    coef_df['AbsCoef'] = np.abs(coef_df['Coefficient'])
    coef_df = coef_df.sort_values('AbsCoef', ascending=False)
    
    # Take top features
    top_coef = coef_df.head(top_n)
    
    # Plot
    plt.figure(figsize=(16, 10))
    colors = ['red' if c < 0 else 'blue' for c in top_coef['Coefficient']]
    sns.barplot(x='Coefficient', y='Feature', data=top_coef, palette=colors)
    
    plt.title(f'Top {top_n} Most Influential Features (Logistic Regression - {year})')
    plt.axvline(x=0, color='gray', linestyle='--')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Feature')
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path)
        print(f"Coefficient plot saved to {output_path}")
    
    plt.close()
    
    return coef_df


Found 4 May files to process:
  - May2021.csv
  - May2022.csv
  - May2023.csv
  - May2024.csv

Loading weather data...
Found 3550 total weather data files
Processed 100 weather files, loaded 0 matching files
Processed 200 weather files, loaded 0 matching files
Processed 300 weather files, loaded 16 matching files
Processed 400 weather files, loaded 32 matching files
Processed 500 weather files, loaded 32 matching files
Processed 600 weather files, loaded 48 matching files
Processed 700 weather files, loaded 48 matching files
Processed 800 weather files, loaded 48 matching files
Processed 900 weather files, loaded 64 matching files
Processed 1000 weather files, loaded 64 matching files
Processed 1100 weather files, loaded 80 matching files
Processed 1200 weather files, loaded 96 matching files
Processed 1300 weather files, loaded 112 matching files
Processed 1400 weather files, loaded 112 matching files
Processed 1500 weather files, loaded 112 matching files
Processed 1600 weather file

In [7]:
# Function to train model
def train_year_model(year, flight_data_file):
    print(f"\n{'='*80}")
    print(f"Training model for year {year}")
    print(f"{'='*80}")

    year_output_dir = os.path.join(output_dir, f'year_{year}')
    os.makedirs(year_output_dir, exist_ok=True)
    os.makedirs(os.path.join(year_output_dir, 'metrics'), exist_ok=True)
    os.makedirs(os.path.join(year_output_dir, 'plots'), exist_ok=True)
    
    start_time = time.time()
    
    # Load and preprocess the year's flight data
    flight_data = load_and_process_flight_data(flight_data_file)
    if flight_data is None or len(flight_data) == 0:
        print(f"No valid flight data available for {year}. Skipping this year.")
        return None

    flight_data = match_weather_data(flight_data)
    
    # Add red-eye flight indicator
    print(f"\nCreating red-eye flight indicator for {year}...")
    flight_data = create_redeye_indicator(flight_data)
    
    # Prepare delay data
    print(f"\nPreparing delay data for {year}...")
    flight_data = prepare_delay_data(flight_data)
    
    # Create time block features
    print(f"\nCreating time block features for {year}...")
    flight_data = create_time_block_features(flight_data)
    
    # Create day features - now handling text day names
    print(f"\nCreating day features for {year}...")
    flight_data = create_day_features(flight_data)
    
    # Feature selection
    print(f"\nSelecting features for delay prediction for {year}...")
    
    # Categorical features
    cat_features = ['DAY_NAME', 'TIME_BLOCK',
                    'MKT_AIRLINE', 'ORIGIN_IATA', 'DEST_IATA', 'EXTREME_WEATHER',
                    'IS_REDEYE', 'IS_WEEKEND', 'IS_MORNING_PEAK', 'IS_EVENING_PEAK']
    
    # Numerical features
    num_features = ['DISTANCE', 'PRCP']
    
    cat_features = [f for f in cat_features if f in flight_data.columns]
    num_features = [f for f in num_features if f in flight_data.columns]
    
    print(f"Using categorical features: {cat_features}")
    print(f"Using numerical features: {num_features}")
    
    # Prepare data for modeling
    X = flight_data[cat_features + num_features].copy()
    y_class = flight_data['IS_DELAYED']
    y_reg = flight_data['DEP_DELAY']
    
    # Handle missing values
    for col in cat_features:
        if X[col].isnull().sum() > 0:
            X[col].fillna('unknown', inplace=True)
    for col in num_features:
        if X[col].isnull().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)
    
    # Split data for classification model
    X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
        X, y_class, test_size=0.1, random_state=2025, stratify=y_class
    )
    
    # Split data for regression model
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X, y_reg, test_size=0.1, random_state=2025
    )
    
    print(f"Training set size: {X_train_class.shape}")
    print(f"Test set size: {X_test_class.shape}")
    
    # Define preprocessing pipeline
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ])
    
    # Train classification model
    print(f"\nTraining delay classification model for {year} (Logistic Regression)...")
    class_model_start_time = time.time()

    class_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            max_iter=1000,
            C=0.1,
            class_weight='balanced',
            random_state=2025,
            n_jobs=-1
        ))
    ])
    
    # Train classification model
    class_model.fit(X_train_class, y_train_class)
    final_class_model = class_model
    
    class_model_training_time = time.time() - class_model_start_time
    print(f"Classification model training took: {class_model_training_time:.2f} seconds")
    
    # Train regression model (Linear Regression)
    print(f"\nTraining delay regression model for {year} (Linear Regression)...")
    reg_model_start_time = time.time()
    
    reg_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression(
            n_jobs=-1
        ))
    ])
    
    # Train regression model
    reg_model.fit(X_train_reg, y_train_reg)
    final_reg_model = reg_model
    
    reg_model_training_time = time.time() - reg_model_start_time
    print(f"Regression model training took: {reg_model_training_time:.2f} seconds")
    
    # Evaluate classification model
    print(f"\nEvaluating delay classification model for {year}...")
    
    y_pred_class = final_class_model.predict(X_test_class)
    y_prob_class = final_class_model.predict_proba(X_test_class)[:, 1]
    
    class_accuracy = (y_pred_class == y_test_class).mean() * 100
    class_roc_auc = roc_auc_score(y_test_class, y_prob_class)

    class_report = classification_report(y_test_class, y_pred_class, output_dict=True)

    class_cm = confusion_matrix(y_test_class, y_pred_class)
    
    print(f"Classification Accuracy: {class_accuracy:.2f}%")
    print(f"Classification ROC AUC: {class_roc_auc:.4f}")
    print(f"Classification Precision (Delayed): {class_report['1']['precision']:.4f}")
    print(f"Classification Recall (Delayed): {class_report['1']['recall']:.4f}")
    print(f"Classification F1 Score (Delayed): {class_report['1']['f1-score']:.4f}")
    
    #Evaluate regression model
    print(f"\nEvaluating delay regression model for {year}...")

    y_pred_reg = final_reg_model.predict(X_test_reg)

    reg_mse = mean_squared_error(y_test_reg, y_pred_reg)
    reg_rmse = np.sqrt(reg_mse)
    reg_mae = mean_absolute_error(y_test_reg, y_pred_reg)
    reg_r2 = r2_score(y_test_reg, y_pred_reg)
    
    print(f"Regression Mean Squared Error: {reg_mse:.2f}")
    print(f"Regression Root Mean Squared Error: {reg_rmse:.2f} minutes")
    print(f"Regression Mean Absolute Error: {reg_mae:.2f} minutes")
    print(f"Regression R² Score: {reg_r2:.4f}")
    
    # Extract model coefficients
    try:
        feature_names = final_class_model.named_steps['preprocessor'].get_feature_names_out()

        log_reg_model = final_class_model.named_steps['classifier']

        coefficients_df = plot_model_coefficients(
            log_reg_model, 
            feature_names,
            year=year,
            top_n=20,
            output_path=os.path.join(year_output_dir, 'plots', f'logistic_regression_coefficients_{year}.png')
        )

        coefficients_df.to_csv(
            os.path.join(year_output_dir, 'metrics', f"logistic_regression_coefficients_{year}.csv"), 
            index=False
        )
        
        # Print top positive and negative coefficients
        print(f"\nTop 5 features increasing delay probability for {year}:")
        print(coefficients_df[coefficients_df['Coefficient'] > 0].head(5))
        
        print(f"\nTop 5 features decreasing delay probability for {year}:")
        print(coefficients_df[coefficients_df['Coefficient'] < 0].head(5))
        
        # Analyze WEEK feature coefficients
        week_features = [f for f in feature_names if 'WEEK' in f]
        if week_features:
            week_coefficients = coefficients_df[coefficients_df['Feature'].isin(week_features)]
            print(f"\nDay of week coefficients for {year} (classification model):")
            print(week_coefficients)
            
            # Save separate plots
            if len(week_coefficients) > 0:
                plt.figure(figsize=(16, 10))
                colors = ['red' if c < 0 else 'blue' for c in week_coefficients['Coefficient']]
                sns.barplot(x='Coefficient', y='Feature', data=week_coefficients, palette=colors)
                plt.title(f'Day of Week Coefficients (Logistic Regression - {year})')
                plt.axvline(x=0, color='gray', linestyle='--')
                plt.xlabel('Coefficient Value (+ increases delay, - decreases delay)')
                plt.ylabel('Day of Week')
                plt.tight_layout()
                plt.savefig(os.path.join(year_output_dir, 'plots', f'day_of_week_coefficients_{year}.png'))
                plt.close()
        
    except Exception as e:
        print(f"Error extracting logistic regression coefficients: {e}")
        coefficients_df = pd.DataFrame()

    try:
        lin_reg_model = final_reg_model.named_steps['regressor']

        lin_coefficients = lin_reg_model.coef_

        lin_coef_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': lin_coefficients
        })

        lin_coef_df['AbsCoef'] = np.abs(lin_coef_df['Coefficient'])
        lin_coef_df = lin_coef_df.sort_values('AbsCoef', ascending=False)

        lin_coef_df.to_csv(
            os.path.join(year_output_dir, 'metrics', f"linear_regression_coefficients_{year}.csv"), 
            index=False
        )
        
        # Plot top linear regression coefficients
        plt.figure(figsize=(16, 10))
        top_coef = lin_coef_df.head(20)
        colors = ['red' if c < 0 else 'blue' for c in top_coef['Coefficient']]
        sns.barplot(x='Coefficient', y='Feature', data=top_coef, palette=colors)
        
        plt.title(f'Top 20 Most Influential Features for Delay Duration (Linear Regression - {year})')
        plt.axvline(x=0, color='gray', linestyle='--')
        plt.xlabel('Coefficient (minutes of delay)')
        plt.ylabel('Feature')
        
        plt.tight_layout()
        plt.savefig(os.path.join(year_output_dir, 'plots', f'linear_regression_coefficients_{year}.png'))
        plt.close()
        
        # Print top features
        print(f"\nTop 5 features increasing delay duration for {year}:")
        print(lin_coef_df[lin_coef_df['Coefficient'] > 0].head(5))
        
        print(f"\nTop 5 features decreasing delay duration for {year}:")
        print(lin_coef_df[lin_coef_df['Coefficient'] < 0].head(5))
        
        if week_features:
            week_lin_coefficients = lin_coef_df[lin_coef_df['Feature'].isin(week_features)]
            print(f"\nDay of week coefficients for {year} (regression model):")
            print(week_lin_coefficients)

            if len(week_lin_coefficients) > 0:
                plt.figure(figsize=(16, 10))
                colors = ['red' if c < 0 else 'blue' for c in week_lin_coefficients['Coefficient']]
                sns.barplot(x='Coefficient', y='Feature', data=week_lin_coefficients, palette=colors)
                plt.title(f'Day of Week Coefficients (Linear Regression - {year})')
                plt.axvline(x=0, color='gray', linestyle='--')
                plt.xlabel('Coefficient Value (minutes of delay)')
                plt.ylabel('Day of Week')
                plt.tight_layout()
                plt.savefig(os.path.join(year_output_dir, 'plots', f'day_of_week_regression_coefficients_{year}.png'))
                plt.close()
        
    except Exception as e:
        print(f"Error extracting linear regression coefficients: {e}")
        lin_coef_df = pd.DataFrame()
    
    # Create visualization plots
    plt.figure(figsize=(16, 10))
    sns.heatmap(class_cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['Not Delayed', 'Delayed'],
               yticklabels=['Not Delayed', 'Delayed'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Delay Classification Confusion Matrix ({year})')
    plt.tight_layout()
    plt.savefig(os.path.join(year_output_dir, 'plots', f'confusion_matrix_{year}.png'))
    plt.close()
    
    # Plot ROC curve for classification model
    plt.figure(figsize=(16, 10))
    fpr, tpr, _ = roc_curve(y_test_class, y_prob_class)
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {class_roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Delay Classification ({year})')
    plt.legend()
    plt.savefig(os.path.join(year_output_dir, 'plots', f'roc_curve_{year}.png'))
    plt.close()

    plt.figure(figsize=(16, 10))
    
    # Create a scatterplot with limited points for clarity
    max_points = 5000
    if len(y_test_reg) > max_points:
        idx = np.random.choice(len(y_test_reg), max_points, replace=False)
        sample_actual = y_test_reg.iloc[idx]
        sample_pred = y_pred_reg[idx]
    else:
        sample_actual = y_test_reg
        sample_pred = y_pred_reg
    
    plt.scatter(sample_actual, sample_pred, alpha=0.3)
    
    max_val = max(sample_actual.max(), sample_pred.max())
    min_val = min(sample_actual.min(), sample_pred.min())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.xlabel('Actual Delay (minutes)')
    plt.ylabel('Predicted Delay (minutes)')
    plt.title(f'Actual vs Predicted Delay ({year})')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(year_output_dir, 'plots', f'actual_vs_predicted_{year}.png'))
    plt.close()
    
    # Plot delay prediction error distribution
    plt.figure(figsize=(16, 10))
    prediction_errors = y_test_reg - y_pred_reg
    sns.histplot(prediction_errors, bins=50, kde=True)
    plt.axvline(0, color='red', linestyle='--')
    plt.xlabel('Prediction Error (minutes)')
    plt.ylabel('Frequency')
    plt.title(f'Delay Prediction Error Distribution ({year})')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(year_output_dir, 'plots', f'error_distribution_{year}.png'))
    plt.close()
    
    # Plot delay by time of day
    if 'TIME_BLOCK' in flight_data.columns:
        plt.figure(figsize=(16, 10))
        time_delay = flight_data.groupby('TIME_BLOCK')['DEP_DELAY'].agg(['mean', 'count']).reset_index()
        time_delay = time_delay.sort_values('mean', ascending=False)

        ax1 = plt.subplot(111)
        bars = sns.barplot(x='TIME_BLOCK', y='mean', data=time_delay, ax=ax1)

        for bar, mean in zip(bars.patches, time_delay['mean']):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{mean:.1f}', ha='center', va='bottom')

        ax2 = ax1.twinx()
        ax2.plot(time_delay.index, time_delay['count'], 'ro-', linewidth=2)

        ax1.set_xlabel('Time Block')
        ax1.set_ylabel('Mean Delay (minutes)')
        ax2.set_ylabel('Number of Flights', color='r')
        plt.title(f'Mean Delay by Time of Day ({year})')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(year_output_dir, 'plots', f'delay_by_time_{year}.png'))
        plt.close()
    
    # Plot delay by day
    if 'WEEK' in flight_data.columns:
        plt.figure(figsize=(16, 10))
        day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        
        week_delay = flight_data.groupby('WEEK')['DEP_DELAY'].mean().reset_index()
        
        week_delay_rate = flight_data.groupby('WEEK')['IS_DELAYED'].mean().reset_index()
        week_delay_rate['IS_DELAYED'] = week_delay_rate['IS_DELAYED'] * 100  # Convert to percentage
        
        if all(day in week_delay['WEEK'].values for day in day_order):
            week_delay['WEEK'] = pd.Categorical(week_delay['WEEK'], categories=day_order, ordered=True)
            week_delay = week_delay.sort_values('WEEK')
            
            week_delay_rate['WEEK'] = pd.Categorical(week_delay_rate['WEEK'], categories=day_order, ordered=True)
            week_delay_rate = week_delay_rate.sort_values('WEEK')
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
        
        bars1 = sns.barplot(x='WEEK', y='DEP_DELAY', data=week_delay, ax=ax1)
        
        for bar, mean in zip(bars1.patches, week_delay['DEP_DELAY']):
            ax1.text(bar.get_x() + bar.get_width()/2, mean + 0.5, 
                    f'{mean:.1f}', ha='center', va='bottom')
        
        ax1.set_xlabel('Day of Week')
        ax1.set_ylabel('Mean Delay (minutes)')
        ax1.set_title(f'Mean Delay by Day of Week ({year})')
        
        # Plot delay rate by day
        bars2 = sns.barplot(x='WEEK', y='IS_DELAYED', data=week_delay_rate, ax=ax2)

        for bar, rate in zip(bars2.patches, week_delay_rate['IS_DELAYED']):
            ax2.text(bar.get_x() + bar.get_width()/2, rate + 0.5, 
                    f'{rate:.1f}%', ha='center', va='bottom')
        
        ax2.set_xlabel('Day of Week')
        ax2.set_ylabel('Delay Rate (%)')
        ax2.set_title(f'Delay Rate by Day of Week ({year})')
        
        plt.tight_layout()
        plt.savefig(os.path.join(year_output_dir, 'plots', f'delay_by_week_{year}.png'))
        plt.close()
        
        # Create a heatmap showing delay patterns by day and hour
        if 'DEP_HOUR' in flight_data.columns:
            plt.figure(figsize=(16, 10))

            delay_heatmap = flight_data.pivot_table(
                values='DEP_DELAY', 
                index='WEEK',
                columns='DEP_HOUR', 
                aggfunc='mean'
            )

            if all(day in delay_heatmap.index for day in day_order):
                delay_heatmap = delay_heatmap.reindex(day_order)

            ax = sns.heatmap(delay_heatmap, cmap='YlOrRd', annot=True, fmt='.1f', linewidths=.5)
            
            plt.title(f'Mean Delay by Day of Week and Hour of Day ({year})')
            plt.xlabel('Hour of Day')
            plt.ylabel('Day of Week')
            plt.tight_layout()
            plt.savefig(os.path.join(year_output_dir, 'plots', f'delay_heatmap_{year}.png'))
            plt.close()

    if 'IS_REDEYE' in flight_data.columns:
        plt.figure(figsize=(16, 10))
        
        redeye_delay = flight_data[flight_data['IS_REDEYE'] == 1]['DEP_DELAY'].mean()
        non_redeye_delay = flight_data[flight_data['IS_REDEYE'] == 0]['DEP_DELAY'].mean()
        
        redeye_count = (flight_data['IS_REDEYE'] == 1).sum()
        non_redeye_count = (flight_data['IS_REDEYE'] == 0).sum()
        
        categories = ['Non-Red-Eye', 'Red-Eye']
        delays = [non_redeye_delay, redeye_delay]
        counts = [non_redeye_count, redeye_count]
        
        bars = plt.bar(categories, delays, color=['skyblue', 'navy'])

        for i, (bar, delay, count) in enumerate(zip(bars, delays, counts)):
            plt.text(i, delay + 0.5, f"{delay:.2f} min\n({count:,} flights)", 
                     ha='center', va='bottom')
        
        plt.ylabel('Mean Delay (minutes)')
        plt.title(f'Mean Delay Comparison: Red-Eye vs. Non-Red-Eye Flights ({year})')
        plt.tight_layout()
        plt.savefig(os.path.join(year_output_dir, 'plots', f'redeye_delay_comparison_{year}.png'))
        plt.close()
    
    # Save models
    dump(final_class_model, os.path.join(year_output_dir, f"logistic_regression_model_{year}.joblib"))
    dump(final_reg_model, os.path.join(year_output_dir, f"linear_regression_model_{year}.joblib"))
    print(f"Models saved to {year_output_dir}")
    
    # Create summary metrics
    metrics = {
        'model_name': f'logistic_regression_{year}',
        'year': year,
        
        # Dataset metrics
        'total_flights': len(flight_data),
        'delayed_flights_rate': flight_data['IS_DELAYED'].mean() * 100,
        'mean_delay': flight_data['DEP_DELAY'].mean(),
        'median_delay': flight_data['DEP_DELAY'].median(),
        'max_delay': flight_data['DEP_DELAY'].max(),
        'min_delay': flight_data['DEP_DELAY'].min(),
        
        # Classification metrics
        'class_accuracy': class_accuracy,
        'class_roc_auc': class_roc_auc,
        'class_precision': class_report['1']['precision'],
        'class_recall': class_report['1']['recall'],
        'class_f1': class_report['1']['f1-score'],
        'class_training_time': class_model_training_time,
        
        # Regression metrics
        'reg_mse': reg_mse,
        'reg_rmse': reg_rmse,
        'reg_mae': reg_mae,
        'reg_r2': reg_r2,
        'reg_training_time': reg_model_training_time,
        
        # Red-eye metrics
        'redeye_count': redeye_count,
        'redeye_percentage': redeye_count / len(flight_data) * 100,
        'redeye_mean_delay': redeye_delay,
        'non_redeye_mean_delay': non_redeye_delay,
        
        'status': 'success',
        'total_processing_time': time.time() - start_time
    }
    
    # Save top important coefficients
    if not coefficients_df.empty:
        top_positive = coefficients_df[coefficients_df['Coefficient'] > 0].head(5)
        for i, (_, row) in enumerate(top_positive.iterrows()):
            metrics[f'top_positive_coef_{i+1}'] = row['Feature']
            metrics[f'top_positive_coef_{i+1}_value'] = float(row['Coefficient'])
        
        top_negative = coefficients_df[coefficients_df['Coefficient'] < 0].head(5)
        for i, (_, row) in enumerate(top_negative.iterrows()):
            metrics[f'top_negative_coef_{i+1}'] = row['Feature']
            metrics[f'top_negative_coef_{i+1}_value'] = float(row['Coefficient'])
        
        if 'week_coefficients' in locals() and not week_coefficients.empty:
            metrics['week_coefficients'] = convert_to_serializable(week_coefficients.to_dict('records'))
    
    if not lin_coef_df.empty:
        top_lin_coef = lin_coef_df.head(10)
        for i, (_, row) in enumerate(top_lin_coef.iterrows()):
            metrics[f'top_linear_coef_{i+1}'] = row['Feature']
            metrics[f'top_linear_coef_{i+1}_value'] = float(row['Coefficient'])
        
        if 'week_lin_coefficients' in locals() and not week_lin_coefficients.empty:
            metrics['week_regression_coefficients'] = convert_to_serializable(week_lin_coefficients.to_dict('records'))
    
    # Save metrics to JSON
    import json
    
    serializable_metrics = convert_to_serializable(metrics)
    with open(os.path.join(year_output_dir, 'metrics', f'model_metrics_{year}.json'), 'w') as f:
        json.dump(serializable_metrics, f, indent=4)
    
    print(f"\nLogistic Regression model training for {year} complete! Total processing time: {metrics['total_processing_time']:.2f} seconds")
    return metrics

In [8]:
# Function to compare models across years
def compare_year_models(all_results):
    print("\nComparing models across years...")
    
    if not all_results or len(all_results) < 2:
        print("Not enough year models to compare.")
        return
    
    # Create a comparison directory
    comparison_dir = os.path.join(output_dir, 'comparison')
    os.makedirs(comparison_dir, exist_ok=True)
    
    # Extract years and sort them
    years = sorted([r['year'] for r in all_results])
    
    # Create DataFrames for different metrics
    class_metrics = pd.DataFrame({
        'Year': years,
        'Accuracy (%)': [r['class_accuracy'] for r in all_results],
        'AUC': [r['class_roc_auc'] for r in all_results],
        'Precision': [r['class_precision'] for r in all_results],
        'Recall': [r['class_recall'] for r in all_results],
        'F1 Score': [r['class_f1'] for r in all_results],
    })
    
    reg_metrics = pd.DataFrame({
        'Year': years,
        'RMSE (min)': [r['reg_rmse'] for r in all_results],
        'MAE (min)': [r['reg_mae'] for r in all_results],
        'R² Score': [r['reg_r2'] for r in all_results],
    })
    
    delay_stats = pd.DataFrame({
        'Year': years,
        'Mean Delay (min)': [r['mean_delay'] for r in all_results],
        'Delay Rate (%)': [r['delayed_flights_rate'] for r in all_results],
        'Total Flights': [r['total_flights'] for r in all_results],
    })
    
    # Plot classification metrics
    plt.figure(figsize=(16, 10))
    
    # Set up bar positions
    bar_width = 0.15
    r1 = np.arange(len(years))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]
    
    # Create bars
    plt.bar(r1, class_metrics['Accuracy (%)'] / 100, width=bar_width, label='Accuracy', color='blue')
    plt.bar(r2, class_metrics['AUC'], width=bar_width, label='AUC', color='green')
    plt.bar(r3, class_metrics['Precision'], width=bar_width, label='Precision', color='red')
    plt.bar(r4, class_metrics['Recall'], width=bar_width, label='Recall', color='purple')
    plt.bar(r5, class_metrics['F1 Score'], width=bar_width, label='F1 Score', color='orange')
    
    # Add texts on bars
    for i, r in enumerate([r1, r2, r3, r4, r5]):
        values = class_metrics.iloc[:, i+1].values
        if i == 0:
            values = values / 100
        for j, v in enumerate(values):
            plt.text(r[j], v + 0.01, f'{v:.2f}' if i > 0 else f'{v*100:.1f}%', 
                    ha='center', va='bottom', rotation=0, fontsize=8)

    plt.xlabel('Year')
    plt.ylabel('Score')
    plt.title('Classification Metrics by Year')
    plt.xticks([r + 2*bar_width for r in range(len(years))], years)
    plt.legend()
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, 'classification_metrics_by_year.png'))
    plt.close()
    
    # Plot regression metrics
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
    
    # Plot RMSE and MAE
    x = np.arange(len(years))
    width = 0.35
    
    ax1.bar(x - width/2, reg_metrics['RMSE (min)'], width, label='RMSE')
    ax1.bar(x + width/2, reg_metrics['MAE (min)'], width, label='MAE')

    for i, v in enumerate(reg_metrics['RMSE (min)']):
        ax1.text(i - width/2, v + 0.5, f'{v:.1f}', ha='center', va='bottom')
    for i, v in enumerate(reg_metrics['MAE (min)']):
        ax1.text(i + width/2, v + 0.5, f'{v:.1f}', ha='center', va='bottom')
    
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Minutes')
    ax1.set_title('Regression Error Metrics')
    ax1.set_xticks(x)
    ax1.set_xticklabels(years)
    ax1.legend()
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Plot R² Score
    bars = ax2.bar(years, reg_metrics['R² Score'], color='green')

    for bar, value in zip(bars, reg_metrics['R² Score']):
        ax2.text(bar.get_x() + bar.get_width()/2, value + 0.01, f'{value:.3f}', 
                ha='center', va='bottom')
    
    ax2.set_xlabel('Year')
    ax2.set_ylabel('R² Score')
    ax2.set_title('Regression R² Score')
    ax2.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, 'regression_metrics_by_year.png'))
    plt.close()
    
    # Plot delay statistics
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))

    bars1 = ax1.bar(years, delay_stats['Mean Delay (min)'], color='blue')

    for bar, value in zip(bars1, delay_stats['Mean Delay (min)']):
        ax1.text(bar.get_x() + bar.get_width()/2, value + 0.3, f'{value:.1f}', 
                ha='center', va='bottom')
    
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Minutes')
    ax1.set_title('Mean Delay by Year')
    ax1.grid(axis='y', linestyle='--', alpha=0.7)

    # Plot delay rate
    bars2 = ax2.bar(years, delay_stats['Delay Rate (%)'], color='red')

    for bar, value in zip(bars2, delay_stats['Delay Rate (%)']):
        ax2.text(bar.get_x() + bar.get_width()/2, value + 0.5, f'{value:.1f}%', 
                ha='center', va='bottom')
    
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Percentage')
    ax2.set_title('Delay Rate by Year')
    ax2.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, 'delay_stats_by_year.png'))
    plt.close()
    
    # Plot total flights
    plt.figure(figsize=(16, 10))
    
    bars = plt.bar(years, delay_stats['Total Flights'], color='purple')

    for bar, value in zip(bars, delay_stats['Total Flights']):
        plt.text(bar.get_x() + bar.get_width()/2, value + 100, f'{value:,}', 
                ha='center', va='bottom')
    
    plt.xlabel('Year')
    plt.ylabel('Number of Flights')
    plt.title('Total Flights by Year')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, 'total_flights_by_year.png'))
    plt.close()
    
    # Create a summary table for all metrics
    summary = pd.concat([
        delay_stats.set_index('Year'),
        class_metrics.set_index('Year').iloc[:, 1:],
        reg_metrics.set_index('Year').iloc[:, 1:]
    ], axis=1)
    
    # Save the summary to CSV
    summary.to_csv(os.path.join(comparison_dir, 'year_model_comparison.csv'))
    print(f"Comparison summary saved to {os.path.join(comparison_dir, 'year_model_comparison.csv')}")
    
    try:
        week_coefficients_by_year = {}
        
        for r in all_results:
            year = r['year']
            if 'week_coefficients' in r:
                week_coefficients_by_year[year] = r['week_coefficients']
        
        if week_coefficients_by_year:
            with open(os.path.join(comparison_dir, 'day_of_week_coefficients_by_year.json'), 'w') as f:
                json.dump(convert_to_serializable(week_coefficients_by_year), f, indent=4)
            
            print(f"Day of week coefficients by year saved to {os.path.join(comparison_dir, 'day_of_week_coefficients_by_year.json')}")
            
            try:
                day_data = []
                
                for year, coeffs in week_coefficients_by_year.items():
                    for coeff in coeffs:
                        if 'Feature' in coeff and 'Coefficient' in coeff:
                            feature = coeff['Feature']
                            if 'WEEK_' in feature:
                                day = feature.split('WEEK_')[1]
                                day_data.append({
                                    'Year': year,
                                    'Day': day,
                                    'Coefficient': coeff['Coefficient']
                                })
                
                if day_data:
                    day_df = pd.DataFrame(day_data)
                    
                    day_pivot = day_df.pivot(index='Day', columns='Year', values='Coefficient')
                    
                    day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
                    day_pivot = day_pivot.reindex([d for d in day_order if d in day_pivot.index])
                    
                    # Plot heatmap
                    plt.figure(figsize=(16, 10))
                    sns.heatmap(day_pivot, cmap='RdBu_r', center=0, annot=True, fmt='.3f')
                    plt.title('Day of Week Coefficients Across Years (Logistic Regression)')
                    plt.xlabel('Year')
                    plt.ylabel('Day of Week')
                    plt.tight_layout()
                    plt.savefig(os.path.join(comparison_dir, 'day_of_week_coefficients_heatmap.png'))
                    plt.close()
                    
                    # Create bar chart showing coefficient evolution for each day
                    plt.figure(figsize=(16, 10))
                    
                    for i, day in enumerate(day_pivot.index):
                        plt.subplot(len(day_pivot.index), 1, i+1)
                        
                        year_vals = day_pivot.loc[day].index.astype(str)
                        coef_vals = day_pivot.loc[day].values
                        
                        bars = plt.bar(year_vals, coef_vals, 
                                      color=['red' if c < 0 else 'blue' for c in coef_vals])

                        for bar, val in zip(bars, coef_vals):
                            plt.text(bar.get_x() + bar.get_width()/2, 
                                    val + (0.01 if val >= 0 else -0.04), 
                                    f'{val:.3f}', ha='center')
                        
                        plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
                        plt.title(f'{day}')
                        plt.ylim(min(day_pivot.values.min() * 1.1, -0.05), 
                                max(day_pivot.values.max() * 1.1, 0.05))

                        if i == len(day_pivot.index) - 1:
                            plt.xlabel('Year')
                    
                    plt.suptitle('Evolution of Day of Week Coefficients Across Years', fontsize=16, y=0.98)
                    plt.tight_layout()
                    plt.subplots_adjust(top=0.93)
                    plt.savefig(os.path.join(comparison_dir, 'day_of_week_coefficients_evolution.png'))
                    plt.close()
            
            except Exception as e:
                print(f"Error creating day of week visualization: {e}")

        feature_comparison = {
            'positive': {},
            'negative': {}
        }
        
        for r in all_results:
            year = r['year']
            feature_comparison['positive'][year] = []
            feature_comparison['negative'][year] = []
            
            # Collect top positive features
            for i in range(1, 6):
                feat_key = f'top_positive_coef_{i}'
                val_key = f'top_positive_coef_{i}_value'
                if feat_key in r and val_key in r:
                    feature_comparison['positive'][year].append({
                        'feature': r[feat_key],
                        'value': r[val_key]
                    })
            
            # Collect top negative features
            for i in range(1, 6):
                feat_key = f'top_negative_coef_{i}'
                val_key = f'top_negative_coef_{i}_value'
                if feat_key in r and val_key in r:
                    feature_comparison['negative'][year].append({
                        'feature': r[feat_key],
                        'value': r[val_key]
                    })
        
        # Save feature comparison to JSON
        with open(os.path.join(comparison_dir, 'feature_coefficient_comparison.json'), 'w') as f:
            json.dump(convert_to_serializable(feature_comparison), f, indent=4)
        
        print(f"Feature coefficient comparison saved to {os.path.join(comparison_dir, 'feature_coefficient_comparison.json')}")
    
    except Exception as e:
        print(f"Error creating feature importance comparison: {e}")
    
    print("Year model comparison completed!")

# Main execution
all_results = []

# Process each year's file separately
for file_path in flight_files:
    year = extract_year_from_filename(file_path)
    results = train_year_model(year, file_path)
    
    if results:
        all_results.append(results)
        print(f"\nModel for year {year} completed successfully!")
    else:
        print(f"\nModel for year {year} failed.")

# After all individual models are trained, compare them
if len(all_results) > 1:
    compare_year_models(all_results)
else:
    print("\nNot enough successful models to perform comparison.")

# Print final summary
print("\nYear-by-Year Model Training Summary:")
for year_result in all_results:
    year = year_result['year']
    print(f"\nYear {year}:")
    print(f"  Total flights: {year_result['total_flights']:,}")
    print(f"  Classification accuracy: {year_result['class_accuracy']:.2f}%")
    print(f"  Classification AUC: {year_result['class_roc_auc']:.4f}")
    print(f"  Regression RMSE: {year_result['reg_rmse']:.2f} minutes")
    print(f"  Regression R²: {year_result['reg_r2']:.4f}")
    print(f"  Mean delay: {year_result['mean_delay']:.2f} minutes")
    print(f"  Delay rate: {year_result['delayed_flights_rate']:.2f}%")
    
    if 'week_coefficients' in year_result:
        print(f"\n  Day of week patterns for {year}:")
        
        week_coeffs = sorted(year_result['week_coefficients'],
                             key=lambda x: x['Coefficient'], 
                             reverse=True)
        
        for coeff in week_coeffs:
            feature = coeff['Feature']
            value = coeff['Coefficient']

            if 'WEEK_' in feature:
                day = feature.split('WEEK_')[1]
                direction = "increases" if value > 0 else "decreases"
                print(f"    • {day}: {direction} delay probability by {abs(value):.4f}")

print("\nTraining complete! Check output directories for detailed results.")


Training model for year 2021

Processing May2021.csv...
Years found in data: [2021]
Months found in data: {5: 520059}
Filtered to only May data: 520059 rows
Filtered from 520059 to 171867 rows for top 30 airports
Removed 485.0 cancelled flights, remaining: 171382
Processing took: 7.59 seconds

Matching weather data with flights...
Processed 10000/171382 rows, matched 7633 flights with weather data
Processed 20000/171382 rows, matched 15255 flights with weather data
Processed 30000/171382 rows, matched 22811 flights with weather data
Processed 40000/171382 rows, matched 30386 flights with weather data
Processed 50000/171382 rows, matched 37926 flights with weather data
Processed 60000/171382 rows, matched 45609 flights with weather data
Processed 70000/171382 rows, matched 53275 flights with weather data
Processed 80000/171382 rows, matched 60864 flights with weather data
Processed 90000/171382 rows, matched 68367 flights with weather data
Processed 100000/171382 rows, matched 75976 fl

<Figure size 1600x1000 with 0 Axes>

<Figure size 1600x1000 with 0 Axes>

<Figure size 1600x1000 with 0 Axes>

<Figure size 1600x1000 with 0 Axes>